<h1><center>DOTA2 winner</center></h1>
    
[Ссылка на соревнование](https://www.kaggle.com/c/mlcourse-dota2-win-prediction)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import pickle
from collections import defaultdict
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit,\
                                    cross_val_score, cross_validate
import scipy.stats as stats
import gc
import warnings
warnings.filterwarnings('ignore')
import os
import category_encoders as ce
from tqdm import tqdm_notebook
import json
from itertools import combinations, groupby
from functools import reduce
from sklearn.cluster import KMeans
import time
from itertools import groupby
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.feature_selection import RFECV
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold



In [2]:
# функция для чтения json файлов
def read_matches(path_to_file):
    with open(path_to_file) as fin:
        for line in fin: 
            yield json.loads(line)

In [3]:
# пути к файлам, размеры файлов
PATH_TR, TOTAL_TR = 'train_matches.jsonl', 39675
PATH_TE, TOTAL_TE = 'test_matches.jsonl', 10000

<img src="признаки.png">

## Целевой признак

In [4]:
target_L = []
idxs = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):    
    target_L.append((MATCH['match_id_hash'], MATCH['targets']['radiant_win']))
    idxs.append(MATCH['match_id_hash'])
y_tr = pd.DataFrame.from_records(target_L).set_index(0)[1].astype(int).values
empty_df = pd.DataFrame(index = idxs)

## Функции для сбора статистики:

* глобальной
* командным сражениям
* целевым объектам
* игрокам

In [5]:
def _collect_globals(PATH_TR, TOTAL_TR):
    
    D1 = defaultdict(list)
    
    # проходим по матчам
    for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):  

        # сохраняем идентификатор матча
        idx = MATCH['match_id_hash']

        # время матча, типа лобби, тип игры
        D1['game_time'].append((idx, MATCH['game_time']))
        D1['game_mode'].append((idx, MATCH['game_mode']))
        D1['lobby_type'].append((idx, MATCH['lobby_type']))

        # чат по командам (длина переписки, интесивность переписки)
        _df = pd.DataFrame.from_records(MATCH['chat'])
        try:
            _df['time'] = _df['time'].abs()
            _df['text'] = _df['text'].apply(len)
            _df['player_slot'] = _df['player_slot'].apply(lambda x: 'r' if x < 100 else 'd')

            chat_len = _df.groupby('player_slot')['text'].sum()
            chat_activity = _df.groupby('player_slot')['time'].apply(lambda x: np.diff(sorted(x)).mean())

            D1['chat'].append((idx, chat_len['r'], chat_len['d'], chat_activity['r'], chat_activity['d']))
        except:
            D1['chat'].append((idx, np.nan, np.nan, np.nan, np.nan))    
    
                
    return D1

def _collect_teamfights(PATH_TR, TOTAL_TR):
    
    D2 = defaultdict(list)
    # проходим по матчам
    for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):  

        # сохраняем идентификатор матча
        idx = MATCH['match_id_hash']        

        # командные сражения (teamfights)        
        # по тимфайтам в матче
        for tf_idx, tf in enumerate(MATCH['teamfights']):

            # начало, конец, время последней смерти, число смертей в тимфайте
            D2['tf_total'].append((idx, tf_idx, tf['start'], tf['end'], tf['last_death'], tf['deaths']))

            # проходим по каждому игроку
            for player_idx, player in enumerate(tf['players']):

                # определяем команду, за которую играет игрок
                team = 'r' if player_idx<=4 else 'd'

                # собираем использованные умения, предметы, убийства
                for key in ('ability_uses', 'item_uses', 'killed'):
                    for k, v in player[key].items():
                        D2['tf_'+key].append((idx, team, k, v))

                # собираем координаты смертей
                for k, v in player['deaths_pos'].items():
                    for k2, v2 in v.items():
                        D2['tf_deaths_pos'].append((idx, team, k, k2, v2))  

                # собираем смерти, выкупы, урон, восстановление здоровья, изменение золота, изменение опыта
                for key in ('deaths', 'buybacks', 'damage', 'healing', 'gold_delta', 'xp_delta'):
                    D2['tf_'+key].append((idx, team, player[key]))
                    
    return D2

def _collect_objectives(PATH_TR, TOTAL_TR):
    
    L =[]
    
    # проходим по матчам
    for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):  

        # сохраняем идентификатор матча
        idx = MATCH['match_id_hash']
        
        # целевые объекты (objectives)    
        for obj in MATCH['objectives']:
            obj['idx'] = idx
            L.append(obj)
            
    return L

def _collect_players(PATH_TR, TOTAL_TR):
    
    D4 = defaultdict(list)
    
    # проходим по матчам
    for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):  

        # сохраняем идентификатор матча
        idx = MATCH['match_id_hash'] 
        
        # игроки
        for player_idx, player in enumerate(MATCH['players']):
            team = 'r' if player_idx<=4 else 'd'

            # числовые статистики
            for key in ['obs_placed', 'sen_placed', 'creeps_stacked', 'camps_stacked', 'rune_pickups', 'firstblood_claimed',
                        'teamfight_participation', 'towers_killed', 'roshans_killed', 'observers_placed', 'stuns',
                        'gold', 'lh', 'xp',  'health', 'max_health', 'max_mana', 'level', 'kills', 'deaths', 'assists',
                        'denies', 'nearby_creep_death_count', 'hero_id', 'x', 'y']:
                D4['player_'+key].append((idx, team, player[key]))

            # айди аккаунта (некоторые игроки - реальные люди,  некоторые - боты)  
            D4['player_account_id_hash'].append((idx, team, player['account_id_hash']))

            # прокачка умений героя
            for upg in player['ability_upgrades']:
                upg['idx'] = idx
                upg['team'] = team
                D4['player_ability_upgrades'].append(upg)

            # использованные умения, предметы
            for key in ('ability_uses', 'item_uses'):
                for k, v in player[key].items():
                    D4['player_'+key].append((idx, team, k, v))     

            # причины получения золота, активность, серии убийств, мульти-убийства, пинги, руны, причины получения опыта
            for key in ['actions', 'gold_reasons', 'kill_streaks', 'life_state', 'multi_kills', 'pings', 'runes', 'xp_reasons']:
                for k, v in player[key].items():
                    D4['player_'+key].append((idx, team, k, v))  

            # логи выкупов
            for bb_log in player['buyback_log']:
                D4['player_buyback_log'].append((idx, 'r' if bb_log['player_slot']<100 else 'd', np.abs(bb_log['time'])))


            for key in ('sen_log', 'obs_log', 'sen_left_log', 'obs_left_log'):
                for log in player[key]:
                    D4['player_'+key].append((idx, 'r' if log['player_slot']<100 else 'd', np.abs(log['time']),\
                                             log['x'], log['y'], log['z']))

            # логи рун
            for log in player['runes_log']:
                log['idx'] = idx
                log['team'] = team
                D4['player_runes_log'].append(log)

            # логи покупок
            for log in player['purchase_log']:
                log['idx'] = idx
                log['team'] = team
                log['time'] = np.abs(log['time'])
                D4['player_purchase_log'].append(log)

            # логи тотемов
            for key in ('obs', 'sen'):
                for k, v in player[key].items():
                    for k2, v2 in v.items():
                        D4['player_'+key].append((idx, team, k, k2, v2))

            # золото, опыт, денаи, ластхиты с течением времени            
            for key in ('gold_t', 'xp_t', 'dn_t', 'lh_t'):
                for ts_idx, ts_value in enumerate(player[key]):
                    D4['player_'+key].append((idx, team, ts_idx, ts_value))

            # покупки
            for k, v in player['purchase'].items():
                D4['player_purchase'].append((idx, team, k, v))

            # инвентарь
            for item in player['hero_inventory']:
                try:
                    D4['player_hero_invenntory'].append((idx, team, item['id'], item['num_charges']))
                except:
                    D4['player_hero_invenntory'].append((idx, team, item['id'], 0))

            # урон
            for k, v in player['damage'].items():
                D4['player_damage'].append((idx, team, k, v))
            
    return D4


## Признаки (глобальная статистика)

In [6]:
D_glob_tr = _collect_globals(PATH_TR, TOTAL_TR)

L_glob = []
for k, v in D_glob_tr.items():
    L_glob.append(empty_df.join(pd.DataFrame.from_records(v).set_index(0)))
X_glob_tr = np.column_stack(L_glob)

del D_glob_tr, L_glob
gc.collect()

4

## Признаки (командные сражения)

In [7]:
D_tf_tr = _collect_teamfights(PATH_TR, TOTAL_TR)

In [8]:
_df = pd.DataFrame.from_records(D_tf_tr['tf_total'])
_df.columns = ['idx', 'tf_idx', 'start', 'end', 'last_death', 'deaths']
_df.loc[:, ['start', 'end', 'last_death']] = _df[['start', 'end', 'last_death']].abs()

L_tf = []
for _idx, _subdf in tqdm_notebook(_df.groupby('idx')):
    
    # число смертей до определенного отрезка матча
    _feats = pd.Series({ts:_subdf[(_subdf['start'] // 60) <= ts]['deaths'].sum() for ts in np.unique(_subdf['start'] // 60)})\
               .to_frame(_idx).T
    
    # средняя длительность сражений
    _feats['duration'] = (_subdf['end'] - _subdf['start']).mean()
    
    # средняя доля отрезка времени с момента последней смерти в длительности сражения 
    _feats['last_death_to_end'] = (_subdf['last_death'] / _subdf['end']).mean()
    
    L_tf.append(_feats)
    
X1_tf_tr = np.column_stack([empty_df.join(_df.groupby('idx')['tf_idx'].nunique().to_frame('nuniques')),\
                            empty_df.join(pd.concat(L_tf))])

del L_tf, _df
gc.collect()

0

In [9]:
bags_abil_item=[]
# мешки использованных умений и айтемов 
for key in tqdm_notebook(('tf_ability_uses', 'tf_item_uses')):
    _df = pd.DataFrame.from_records(D_tf_tr[key])
    _df[4] = _df[1].replace({'r':1, 'd':-1}) * _df[3]
    bags_abil_item.append(empty_df.join(_df.groupby([0, 2])[4].sum().unstack()))
    bags_abil_item.append(empty_df.join(_df.groupby([0, 2])[4].mean().unstack()))
    
X2_tf_tr = np.column_stack(bags_abil_item)

del bags_abil_item, _df
gc.collect()

3

In [10]:
counts_L = []
for key in tqdm_notebook(('tf_deaths', 'tf_buybacks', 'tf_damage',\
                          'tf_healing', 'tf_gold_delta', 'tf_xp_delta')):
    _df = pd.DataFrame.from_records(D_tf_tr[key])
    _df[3] = _df[1].replace({'r':1, 'd':-1}) * _df[2]
    counts_L.append(empty_df.join(_df.groupby(0)[3].sum().to_frame()))
    
X3_tf_tr = np.column_stack(counts_L)

del counts_L, _df
gc.collect()

3

In [11]:
_df = pd.DataFrame.from_records(D_tf_tr['tf_deaths_pos'])

_df['xy'] = (_df[2].astype(int)/10).round().astype(int).astype(str) +'_'+\
            (_df[3].astype(int)/10).round().astype(int).astype(str)

X4_tf_tr = np.column_stack([empty_df.join(_df.groupby([0, 'xy'])[4].sum().unstack()),\
                            empty_df.join(_df.groupby([0, 'xy'])[4].mean().unstack())])

del _df
gc.collect()

0

In [12]:
del D_tf_tr
gc.collect()

20

## Признаки (целевые объекты)

In [13]:
L_obj_tr = _collect_objectives(PATH_TR, TOTAL_TR)

_df = pd.DataFrame.from_records(L_obj_tr)

_df.loc[_df['team'].isna(), 'flag'] = _df[_df['team'].isna()]['player_slot'].apply(lambda x: 2 if x<100 else 3)
_df.loc[~_df['team'].isna(), 'flag'] = _df[~_df['team'].isna()]['team']
_df.loc[_df['flag'].isin([60, 100]), 'flag'] = _df[_df['flag'].isin([60, 100])]['slot'].apply(lambda x: 2 if x<=4 else 3)
_df['flag'] = _df['flag'].replace({2:1, 3:-1})
_df['time'] = _df['time'].abs()

L = []
L.append(empty_df.join(_df.groupby(['idx', 'type'])['flag'].sum().unstack()))
L.append(empty_df.join(_df.groupby(['idx', 'type'])['flag'].mean().unstack()))

X_obj_tr = np.column_stack(L)

del L_obj_tr, L, _df
gc.collect()

4

## Признаки (игроки)

In [14]:
D_player_tr = _collect_players(PATH_TR, TOTAL_TR)

In [15]:
L = []
for key in tqdm_notebook(['player_obs_placed', 'player_sen_placed', 'player_creeps_stacked', 'player_camps_stacked',\
                          'player_rune_pickups', 'player_firstblood_claimed',
                          'player_teamfight_participation', 'player_towers_killed',\
                          'player_roshans_killed', 'player_observers_placed', 'player_stuns',
                          'player_gold', 'player_lh', 'player_xp',  'player_health', 'player_max_health',\
                          'player_max_mana', 'player_level', 'player_kills', 'player_deaths', 'player_assists',
                          'player_denies', 'player_nearby_creep_death_count']):
    _df = pd.DataFrame.from_records(D_player_tr[key])
    _df[3] = _df[1].replace({'r':1, 'd':-1}) * _df[2]
    L.append(empty_df.join(_df.groupby(0)[3].sum().to_frame()))
    L.append(empty_df.join(_df.groupby(0)[3].mean().to_frame()))
    
for key in tqdm_notebook(['player_hero_id', 'player_x', 'player_y']):
    _df = pd.DataFrame.from_records(D_player_tr[key])
    _df[1] = _df[1].replace({'r':1, 'd':-1})
    L.append(empty_df.join(_df.groupby([0, 2])[1].sum().unstack()))

_df= pd.DataFrame.from_records(D_player_tr['player_ability_upgrades'])
L.append(empty_df.join(_df.groupby(['idx', 'team'])['time'].apply(lambda x: np.diff(sorted(x)).mean()).unstack()))

for key in tqdm_notebook(('player_ability_uses', 'player_item_uses')):
    _df= pd.DataFrame.from_records(D_player_tr[key])
    _df[4] = _df[1].replace({'r':1,'d':-1})* _df[3]
    L.append(empty_df.join(_df.groupby([0,2])[4].sum().unstack()))
    L.append(empty_df.join(_df.groupby([0,2])[4].mean().unstack()))  
    
for key in tqdm_notebook(['player_actions', 'player_gold_reasons', 'player_kill_streaks',\
            'player_life_state', 'player_multi_kills', 'player_pings',\
            'player_runes', 'player_xp_reasons']):
    _df= pd.DataFrame.from_records(D_player_tr[key])
    _df[4] = _df[3]*_df[1].replace({'r':1, 'd':-1})
    L.append(empty_df.join(_df.groupby([0, 2])[4].sum().unstack()))
    L.append(empty_df.join(_df.groupby([0, 2])[4].mean().unstack()))
    
_df= pd.DataFrame.from_records(D_player_tr['player_buyback_log'])
L.append(empty_df.join(_df.groupby([0,1])[2].apply(lambda x: np.diff(sorted(x)).mean()).unstack()))

_df= pd.DataFrame.from_records(D_player_tr['player_runes_log'])
L2,L3, L4=[], [], []
for _idx, _subdf in tqdm_notebook(_df.groupby('idx')):
    _subdf['team2']= _subdf['team'].replace({'r':1,'d':-1})
    r_df= _subdf[_subdf['team']=='r']
    d_df = _subdf[_subdf['team']=='d']
    
    L2.append(_subdf.groupby(['idx', 'key'])['team2'].sum().unstack())
    L3.append(_subdf.groupby(['idx', 'key'])['team2'].mean().unstack())
    L4.append((_idx,np.diff(sorted(r_df['time'])).mean()-np.diff(sorted(d_df['time'])).mean()))
    
L.append(empty_df.join(pd.concat(L2)))
L.append(empty_df.join(pd.concat(L3)))
L.append(empty_df.join(pd.DataFrame.from_records(L4).set_index(0)))

for key in tqdm_notebook(('sen_log', 'obs_log', 'sen_left_log', 'obs_left_log')):
    _df= pd.DataFrame.from_records(D_player_tr['player_'+key])
    coords_df =(_df[[3, 4, 5]]/10).round().astype(int).astype(str)
    _df['xyz'] = coords_df[3]+'_'+coords_df[4]+'_'+coords_df[5]
    _df[6] = _df[1].replace({'r':1,'d':-1})
    L.append(empty_df.join(_df.groupby([0, 'xyz'])[6].sum().unstack()))
    L.append(empty_df.join(_df.groupby([0, 'xyz'])[6].mean().unstack()))

_df= pd.DataFrame.from_records(D_player_tr['player_damage'])
_df[4] = _df[1].replace({'r':1,'d':-1}) *_df[3]
L.append(empty_df.join(_df.groupby([0, 2])[4].sum().unstack()))
L.append(empty_df.join(_df.groupby([0, 2])[4].mean().unstack()))     

X_player_tr = pd.concat(L,1)

In [16]:
X_tr = np.column_stack([X_glob_tr, X1_tf_tr, X2_tf_tr, X3_tf_tr, X4_tf_tr, X_obj_tr, X_player_tr])
del X_glob_tr, X1_tf_tr, X2_tf_tr, X3_tf_tr, X4_tf_tr, X_obj_tr, X_player_tr
gc.collect()

20

In [17]:
print('Число признаков = {}'.format(X_tr.shape[1]))

Число признаков = 5059


In [18]:
class FeatureSelector():
    def __init__(self, estimator,
                       metric,\
                       larger_is_better,\
                       cv, 
                       use_values,
                       use_recursion,
                       fill_na,\
                       show_progress, 
                       early_stopping = None):
        '''
        Инициализирует модель для отбора признаков
        
        Параметры:
            1) estimator - модель
            2) metric - метрика качества (названия метрик sklearn + может быть кастомная)
            3) larger_is_better - критерий оптимизации (чем больше, тем лучше)
            4) cv - схема валидации
            5) use_values - индексы столбцов, в которых требуется отобрать значения
            6) use_recursion - использовать рекурсию в отборе
            7) fill_na - значение, которым заполняются np.nan
            8) show_progress - печатать результаты валидации
            9) early_stopping - число итераций без улучшения метрики для ранней остановки отбора
        Возвращает:
            1) fit - производит отбор признаков
            2) transform - оставляет отобранные признаки
            3) return_self - возвращает 
                - best_features - отобранные признаки(список)
                - D_best_features - отобранные значения признаков (словарь: {признак:значения})
                - best_score - лучшее значение метрики
        '''
        self.estimator = estimator
        self.metric = metric
        self.cv = cv
        self.use_values = use_values        
        self.use_recursion = use_recursion
        self.show_progress = show_progress
        self.early_stopping = early_stopping
        self.fill_na = fill_na
        self.larger_is_better = larger_is_better
    def fit(self, X, Y):        
        flag = isinstance(X[:, 0], csc_matrix)
        # список с результатами валидации
        column_value_score = []
        # проходим по признакам
        for i in tqdm_notebook(range(X.shape[1])):
            # если формат матрицы признаков == csc_matrix
            if flag:
                # выбираем столбец, преобразуем
                ser = pd.DataFrame(X[:, i].todense())[0].values.flatten()
            # если формат != csc_matrix
            else:
                # выбираем столбец
                ser = X[:, i]        
            # если столбец в списке с проверкой значений 
            if self.use_values is not None:                
                if i in self.use_values:
                    # уникальные значения столбца
                    unique_values = np.unique(ser)  
                    # валидируем каждое значение
                    for val in unique_values:
                        _x = np.int32(ser==val).reshape(-1,1)
                        column_value_score.append((i, val,\
                                                   cross_val_score(self.estimator,\
                                                                   _x, Y,\
                                                                   scoring = self.metric,\
                                                                   cv = self.cv).mean()))
                else: 
                    # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               _x, Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
            else:
                # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               ser.reshape(-1,1), Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
                

        # признаки и значения признаков в порядке убывания валидации
        order = np.array(sorted(column_value_score, key = lambda x: x[-1], reverse = True))[:, :2]             
        # список лучших признаков
        best_features = []
        # словарь лучших значений признаков
        D_best_features = defaultdict(list)
        # список с признаками, не давшими прироста
        to_drop = []
        
        # лучшее значение метрики
        if self.larger_is_better:
            best_score = 0
        else:
            best_score = np.inf            
        counter = 0
        # проходим по признакам и значениям признаков в порядке убывания валидации
        for feature, feature_value in tqdm_notebook(order):   

            # добавляем текущие признаки/значения
            if feature_value is None:
                best_features.append(feature)               
            else:
                D_best_features[feature].append(feature_value)

            # обновляем матрицы
            L = []
            for k, v in D_best_features.items():
                if isinstance(X[:, k], csc_matrix):
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

            if flag:
                if (len(best_features)>0) & (len(L)>0):
                    _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                elif (len(best_features)==0) & (len(L)>0):
                    _X = csc_matrix(np.column_stack(L))
                elif (len(best_features)>0) & (len(L)==0):
                    _X = csc_matrix(X[:, best_features])                    
                        
            else:
                if (len(best_features)>0) & (len(L)>0):
                    _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                elif (len(best_features)==0) & (len(L)>0):
                    _X = np.column_stack(L)
                elif (len(best_features)>0) & (len(L)==0):
                    _X = X[:, best_features] 
            # считаем валидацию    
            current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
            # если метрика улучшилась
            if self.larger_is_better:
                if current_score>best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break
            else:
                if current_score<best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                    # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break

        if self.use_recursion:
            # запускаем бесконечный цикл
            while True:
                # списки лучших признаков до и после
                to_drop_before = to_drop
                to_drop_after = []
                # проходим по признакам и значениям признаков в порядке убывания валидации
                for feature, feature_value in tqdm_notebook(to_drop_before):   
                    # добавляем текущие признаки/значения
                    if feature_value is None:
                        best_features.append(feature)               
                    else:
                        D_best_features[feature].append(feature_value)

                    # обновляем матрицы
                    L = []
                    for k, v in D_best_features.items():
                        if isinstance(X[:, k], csc_matrix):
                            L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                        else:
                            L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

                    if flag:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = csc_matrix(np.column_stack(L))
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = csc_matrix(X[:, best_features])                    

                    else:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = np.column_stack(L)
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = X[:, best_features] 

                    # считаем валидацию    
                    current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
                    
                    if self.larger_is_better:
                        if current_score>best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                            # если метрика не улучшилась
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    else:
                        if current_score<best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    

                # если списки одинаковые, останавливаем отбор
                if len(to_drop_after) == len(to_drop_before):
                    break
                # если разные - обновляем списки до и после
                else:
                    to_drop_before = to_drop_after
                    to_drop_after = []
                    
        self.best_features = best_features
        self.D_best_features = D_best_features
        self.best_score =best_score
        self.flag = flag
    def transform(self, X):
              
        if len(self.best_features) !=0:
            x1 = X[:, self.best_features]
        else:
            x1 = None
        if len(list(self.D_best_features.keys())) !=0:
            L=[]
            for k, v in self.D_best_features.items():
                if self.flag:
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))                    
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))
            x2 = np.column_stack(L)
        else:
            x2 = None
            
        if (x1 is not None) & (x2 is not None):
            if self.flag: 
                _X = csc_matrix(hstack([x1, x2]))
            else:
                _X = np.column_stack([x1, x2])
                
        if (x1 is not None) & (x2 is None):
            _X = x1
        if (x1 is None) & (x2 is not None):
            if self.flag:
                _X = csc_matrix(x2)
            else:
                _X = x2
        return _X     
        
    def return_self(self):
        return self

In [19]:
from scipy.sparse import csc_matrix, hstack

In [21]:
SEED=13
X_tr = pd.DataFrame(X_tr).replace({np.inf:-1}).fillna(-1).values
scaler = MinMaxScaler()
X_scaled_tr = scaler.fit_transform(X_tr)

X_TR, X_HOLDOUT, Y_TR, Y_HOLDOUT = train_test_split(X_scaled_tr, y_tr, stratify = y_tr, test_size= .1, random_state = SEED)
skf3= StratifiedKFold(3,random_state = SEED)
logit_clf = LogisticRegression(random_state = SEED)
lgb_clf = LGBMClassifier(random_state = SEED)

In [26]:
selector = FeatureSelector(estimator = lgb_clf,\
                          metric = 'roc_auc',\
                          larger_is_better = True,\
                          cv = skf3,\
                          use_values = None,\
                          use_recursion = False,\
                          fill_na = -1,\
                          show_progress = False, 
                          early_stopping = None)            
selector.fit(X_TR, Y_TR)     
X_sel_scaled_tr = selector.transform(X_scaled_tr)
X_sel_TR, X_sel_HOLD = selector.transform(X_TR), selector.transform(X_HOLDOUT)

In [27]:
# лучший auc
best_score = selector.return_self().best_score

#### смешиваем предсказания моделей

In [34]:
# вес1
for w1 in tqdm_notebook(np.linspace(0, 1, 100)):
    
    # вес2
    w2 = 1-w1
    aucs = []
    
    # запускаем валидацию
    for tr_idx, val_idx in skf3.split(X_sel_TR, Y_TR):
        
        xtr, xval = X_sel_TR[tr_idx], X_sel_TR[val_idx]
        ytr, yval = Y_TR[tr_idx], Y_TR[val_idx]
        
        # фитим модели
        lgb_clf.fit(xtr,ytr)
        logit_clf.fit(xtr,ytr)

        # делаем предсказания
        lgb_predprob = lgb_clf.predict_proba(xval)[:, 1]
        logit_predprob = logit_clf.predict_proba(xval)[:, 1]   
        
        # смешиваем предсказания
        y_blend = lgb_predprob*w1 + logit_predprob*w2
        
        # сохраняем метрику
        aucs.append(roc_auc_score(yval, y_blend))
        
    # средняя метрика смешанных предсказаний
    mean_score = np.mean(aucs)
    
    # если метрика смешанных предскзаний лучше метрики бустинга
    if mean_score > best_score:
        best_score = mean_score
        best_w1 = w1
        print('best score blend = {:.5f}'.format(best_score))
        
    

best score blend = 0.83339
best score blend = 0.83345
best score blend = 0.83349
best score blend = 0.83354
best score blend = 0.83358
best score blend = 0.83363
best score blend = 0.83366
best score blend = 0.83369
best score blend = 0.83372
best score blend = 0.83375
best score blend = 0.83377
best score blend = 0.83379
best score blend = 0.83381
best score blend = 0.83383
best score blend = 0.83384
best score blend = 0.83385
best score blend = 0.83386
best score blend = 0.83386
best score blend = 0.83387
best score blend = 0.83387



In [37]:
print('вес lgb = {:.2f}, вес logit = {:.2f}'.format(best_w1, 1-best_w1))

вес lgb = 0.79, вес logit = 0.21
